In [7]:
from president_speech.db.parquet_interpreter import read_parquet
import pandas as pd

In [8]:
df = read_parquet()

In [9]:
df.head(1)

,division_number,president,title,date,location,kind,speech_text
5368,1305368,박정희,제5대 대통령 취임식 대통령 취임사,1963.12.17,국내,취임사,"\n\n\n단군성조가 천혜의 이 강토 위에 국기를 닦으신지 반만년, 연면히 이어온 ..."


## 🔍 검색 단어 입력 - search_word
###  ⬇ 아래에서 계속 사용합니다 („• ֊ •„)੭

In [22]:
search_word = "자유"

In [23]:
def add_keyword_count(df: pd.DataFrame, keyword: str) -> pd.DataFrame:
    """
    DataFrame에 keyword_count 컬럼을 추가하여 반환합니다.
    각 speech_text에서 keyword가 등장하는 횟수를 계산합니다.
    """
    # keyword_count 컬럼 추가
    df['keyword_count'] = df['speech_text'].str.count(keyword)
    return df

In [32]:
df_kc = add_keyword_count(df, search_word)

In [33]:
df_kc.columns

Index(['division_number', 'president', 'title', 'date', 'location', 'kind',
       'speech_text', 'keyword_count'],
      dtype='object')

In [34]:
df_kc.dtypes

division_number     int64
president          object
title              object
date               object
location           object
kind               object
speech_text        object
keyword_count       int64
dtype: object

In [35]:
# („• ֊ •„)੭ 연설문 당 search_word 가 speech_text 에 몇번 있는지 카운트 된 결과 입니다
df_kc[['president','title','keyword_count']].sample(frac=1, random_state=42).head(3)

,president,title,keyword_count
101499,문재인,시스템 반도체 비전선포식,0
6422,박정희,아메리칸 대학의 제7차 인터내셔널 위크에 보내는 메시지,0
8804,김대중,6·25 제49주년 참전용사 위로연 연설(한반도 평화와 번영을 위한 호국정신의 계승),11


In [45]:

f_df = df_kc[df_kc['keyword_count'] >= 10]

# random_state는 랜덤한 결과를 재현 가능하게 만들기 위한 파라미터입니다. 
# 즉, random_state 값을 설정하면 
# 같은 데이터와 같은 조건에서 언제든지 동일한 랜덤 결과를 얻을 수 있습니다. 
# 이를 통해 실험을 반복하거나 다른 사람과 작업을 공유할 때 결과가 일관되게 유지됩니다.
# 42(은하수를 여행하는 히치하이커를 위한 안내서) 에서옴 - 아무 숫자나 가능
f_df[['president', 'title', 'keyword_count']].sample(frac=1, random_state=42).head(3)

,president,title,keyword_count
10997,이승만,상항(桑港) 콤몬웰스 클라브에서의 연설,13
6550,박정희,제9차 세계 반공 연맹 총회 치사,10
11198,이승만,공군본부 신청사 낙성식에 치사,13


In [50]:
a_df = f_df.groupby("president").agg(
    count=("speech_text", "size"),  # 연설 개수
    keyword_sum=("keyword_count", "sum")  # keyword 발생 횟수 합산
)

In [52]:
s_df = a_df.sort_values(by=["keyword_sum", "count"], ascending=[False, False])

In [53]:
s_df.reset_index()

,president,count,keyword_sum
0,이승만,59,881
1,박정희,47,757
2,노태우,23,314
3,노무현,6,129
4,김대중,6,110
5,전두환,4,67
6,이명박,4,66
7,문재인,5,61
8,김영삼,3,45


In [57]:
f_df.groupby("president")["keyword_count"].sum().reset_index()

,president,keyword_count
0,김대중,110
1,김영삼,45
2,노무현,129
3,노태우,314
4,문재인,61
5,박정희,757
6,이명박,66
7,이승만,881
8,전두환,67
